# Proyecto de visualización

## Sistemas de calefacción (incluye bombas de calor)

In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path



## Consumo energético residencial (por uso)

## Clima (para HDD/CDD) — 

la vía realmente abierta

https://opendatadocs.meteoswiss.ch/

https://opendatadocs.meteoswiss.ch/a-data-groundbased/a1-automatic-weather-stations#data-download

In [1]:
import requests
import pandas as pd

COLL = "ch.meteoschweiz.ogd-smn"
BASE = "https://data.geo.admin.ch/api/stac/v1"

def stac_search(collection=COLL, limit=100, **kwargs):
    url = f"{BASE}/search"
    payload = {"collections": [collection], "limit": limit, **kwargs}
    r = requests.post(url, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()

# 1) Trae algunos items (primeras páginas) para inspeccionar qué campos trae cada item
data = stac_search(limit=50)
items = data["features"]
items[0].keys(), items[0]["properties"].keys()

(dict_keys(['id', 'collection', 'type', 'stac_version', 'geometry', 'bbox', 'properties', 'stac_extensions', 'links', 'assets']),
 dict_keys(['datetime', 'title', 'created', 'updated']))

In [2]:
def find_daily_historical_item(items, station_code=None):
    matches = []
    for it in items:
        p = it["properties"]
        # Ajusta estas condiciones según lo que veas en properties
        ok_station = True if station_code is None else (p.get("station_code") == station_code or p.get("station") == station_code)
        ok_daily = "daily" in str(p).lower() or p.get("time_resolution") == "d"
        ok_hist = "historical" in str(p).lower() or p.get("update_interval") == "historical"
        if ok_station and ok_daily and ok_hist:
            matches.append(it)
    return matches

matches = find_daily_historical_item(items, station_code="SMA")  # ejemplo: cambia el código
len(matches)


0

In [3]:
def get_csv_url_from_item(item):
    # Suele haber un asset con key "data" o similar. Inspecciona item["assets"].keys()
    assets = item["assets"]
    # intenta detectar automáticamente un CSV
    for k, a in assets.items():
        href = a.get("href", "")
        if href.lower().endswith(".csv"):
            return href
    raise ValueError("No encontré un asset CSV en este item.")

csv_url = get_csv_url_from_item(matches[0])
df = pd.read_csv(csv_url)
df.head(), df.columns[:10]


IndexError: list index out of range

In [4]:
def get_csv_url_from_item(item):
    # Suele haber un asset con key "data" o similar. Inspecciona item["assets"].keys()
    assets = item["assets"]
    # intenta detectar automáticamente un CSV
    for k, a in assets.items():
        href = a.get("href", "")
        if href.lower().endswith(".csv"):
            return href
    raise ValueError("No encontré un asset CSV en este item.")

csv_url = get_csv_url_from_item(matches[0])
df = pd.read_csv(csv_url)
df.head(), df.columns[:10]


IndexError: list index out of range

In [ ]:
import numpy as np

BASE_H = 18.0
BASE_C = 24.0

# Supongamos que ya tienes:
# df["date"] con fechas y df["Tmean"] con temperatura media diaria

df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year

df["HDD"] = np.maximum(0, BASE_H - df["Tmean"])
df["CDD"] = np.maximum(0, df["Tmean"] - BASE_C)

annual = df.groupby("year")[["HDD", "CDD"]].sum().reset_index()
annual.head()
